# Лабораторная работ №3 - "Подбор гиперпараметров модели" (ИНДИВИДУАЛЬНАЯ)

---

##### *Выполнил*: Крушинин Н.Г.
##### *Группа*: М8О-307Б-23
##### *Датасет (ссылка)*: [\*тык\*](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)

---

# Задание:

0. Выбрать модель для обучения (Desicion tree, Random forest, SVM, KNN, Boosting)
1. Показать какие гиперпараметры есть у выбранной модели (В виде таблички).
2. Выбрать датасет для обучения и взависимости от модели подготовить данные
3. Подобрать гиперпараметры для модели и сравнить лучшие подборы, для (Grid Search, RandomSearch, Optuna)
4. На самом лучшем обучении (Grid Search, RandomSearch, Optuna) сделать калькулятор, который показывает `локальную интерпретацию` с помощью [LIME](https://habr.com/ru/companies/otus/articles/779430/) и `глобальную интерпретацию` с помощью [SHAP](https://habr.com/ru/companies/ods/articles/599573/).

    [Доп статья](https://habr.com/ru/companies/wunderfund/articles/739744/)

# 0. Глобальная настройка

In [ ]:
RND_SEED = 42


# 1. Подготовка данных

In [ ]:
import kagglehub
import pandas as pd
from kagglehub import KaggleDatasetAdapter

file_path = "creditcard.csv"

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "mlg-ulb/creditcardfraud",
    file_path,
)


In [ ]:
df.head(10)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [ ]:
from sklearn.model_selection import train_test_split


X, y = df.drop("Class", axis=1), df["Class"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RND_SEED)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


model_forest = RandomForestClassifier(
    n_estimators=100,
    criterion="gini",
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=RND_SEED,
    n_jobs=-1,
    class_weight="balanced",
    min_impurity_decrease=0.0
)


pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", model_forest)
])

pipeline


,steps,"[('scaler', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2


In [ ]:
param_grid = {
    "classifier__criterion": ["entropy"],
    "classifier__max_depth": [None, 5],
    "classifier__min_samples_split": [2, 10],
    "classifier__min_samples_leaf": [1, 4],
    "classifier__max_features": ["sqrt", 0.5]
}


# 2.1. Grid Search

In [ ]:
import os
import warnings

# Перед любыми fit()/GridSearch — чтобы унаследовали подпроцессы joblib
os.environ["PYTHONWARNINGS"] = "ignore:::pkg_resources"

# На всякий случай приглушим и на уровне интерпретатора
warnings.filterwarnings("ignore", category=UserWarning, module="pkg_resources")


In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RND_SEED)

# ── Меняем только параметры под дисбаланс: метрики и refit ──
gs_tree = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring={"pr": "average_precision", "roc": "roc_auc"},
    refit="pr",
    cv=cv,
    n_jobs=-1,
    verbose=3,
    return_train_score=True
)

gs_tree.fit(X_train, y_train)

print("\nЛучшие параметры:")
for key, value in gs_tree.best_params_.items():
    print(f"{key} -> {value}")
print("\nЛучшая средняя PR-AUC (CV):",
      gs_tree.cv_results_["mean_test_pr"][gs_tree.best_index_])
print("Соответствующий ROC-AUC (CV):",
      gs_tree.cv_results_["mean_test_roc"][gs_tree.best_index_])

print("Точность на тесте:", gs_tree.score(X_test, y_test))


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 3/5] END classifier__criterion=gini, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2; pr: (train=1.000, test=0.866) roc: (train=1.000, test=0.961) total time= 1.5min
[CV 3/5] END classifier__criterion=gini, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=4, classifier__min_samples_split=2; pr: (train=0.985, test=0.860) roc: (train=1.000, test=0.953) total time= 1.5min
[CV 1/5] END classifier__criterion=gini, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2; pr: (train=1.000, test=0.884) roc: (train=1.000, test=0.961) total time= 1.5min
[CV 2/5] END classifier__criterion=gini, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=10; pr: (train=0.997, test=0.823) roc: (t

# 2.2. Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=16,
    scoring={"pr": "average_precision", "roc": "roc_auc"},
    refit="pr",
    cv=cv,
    n_jobs=-1,
    verbose=3,
    return_train_score=True
)

random_search.fit(X_train, y_train)

# Результаты
print("\nЛучшие параметры:")
for key, value in random_search.best_params_.items():
    print(f'{key} -> {value}')
print("\nЛучшее качество на кросс-валидации:", random_search.best_score_)
print("Точность на тесте:", random_search.score(X_test, y_test))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 3/5] END classifier__criterion=entropy, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=4, classifier__min_samples_split=2; pr: (train=0.990, test=0.858) roc: (train=1.000, test=0.966) total time= 1.1min
[CV 1/5] END classifier__criterion=entropy, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2; pr: (train=1.000, test=0.888) roc: (train=1.000, test=0.960) total time= 1.1min
[CV 3/5] END classifier__criterion=entropy, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=10; pr: (train=0.998, test=0.862) roc: (train=1.000, test=0.960) total time= 1.1min
[CV 5/5] END classifier__criterion=entropy, classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=10; pr: (train=0.997, test=0.

# 2.3. Optuna

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score


optuna.logging.disable_default_handler()


def objective_dt(trial):
    max_depth = trial.suggest_int('classifier__max_depth', 1, 5)
    min_samples_split = trial.suggest_int(
        'classifier__min_samples_split', 2, 4)
    min_samples_leaf = trial.suggest_int('classifier__min_samples_leaf', 1, 4)
    criterion = trial.suggest_categorical(
        'classifier__criterion', ['entropy'])

    pipeline.set_params(
        classifier__max_depth=max_depth,
        classifier__min_samples_split=min_samples_split,
        classifier__min_samples_leaf=min_samples_leaf,
        classifier__criterion=criterion
    )

    score = cross_val_score(pipeline, X_train, y_train,
                            cv=cv, scoring="average_precision").mean()
    return score


# Создаём исследование
study_dt = optuna.create_study(
    direction="maximize"  # максимизирукм cross_val_score чтобы он был наибольшим
)

study_dt.optimize(
    objective_dt,
    n_trials=60,  # количество попыток
    n_jobs=-1,
)


# Лучшие параметры
print("\nЛучшие параметры:")
for key, value in study_dt.best_params.items():
    print(f'{key} -> {value}')

pipeline.set_params(**study_dt.best_params)
pipeline.fit(X_train, y_train)
print("\nЛучшее качество на кросс-валидации:", cross_val_score(pipeline,
      X_train, y_train, cv=cv, scoring="average_precision").mean())
print("Точность на тесте:", pipeline.score(X_test, y_test))



Лучшие параметры:
classifier__max_depth -> 5
classifier__min_samples_split -> 3
classifier__min_samples_leaf -> 3
classifier__criterion -> entropy

Лучшее качество на кросс-валидации: 0.7946922865296099
Точность на тесте: 0.9983322214809873


# 2.4. TPOT

In [ ]:
os.environ["PYTHONWARNINGS"] = "ignore:::pkg_resources"

# На всякий случай приглушим и на уровне интерпретатора
warnings.filterwarnings("ignore", category=UserWarning, module="pkg_resources")


In [ ]:
from tpot import TPOTClassifier
from sklearn.metrics import precision_score


# Создаём TPOT-классификатор
tpot = TPOTClassifier(
    generations=3,        # количество поколений эволюций
    population_size=10,   # размер популяции
    verbose=2,            # уровень детализации вывода
    random_state=RND_SEED,
    cv=5,                 # количество фолдов кросс-валидации
)


In [ ]:
warnings.filterwarnings(
    "ignore",
    message="Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.",
    category=UserWarning,
    module="tpot"
)


In [ ]:
import os
import warnings
os.environ["PYTHONWARNINGS"] = "ignore"
warnings.simplefilter("ignore")


In [ ]:
from joblib import dump
# Обучаем
print("Начало оптимизации с TPOT...")
tpot.fit(X_train, y_train)


dump(tpot.fitted_pipeline_, "tpot_pipeline.joblib")
print("Сохранено: tpot_pipeline.joblib")


Начало оптимизации с TPOT...


Generation: 100%|██████████| 3/3 [47:47<00:00, 955.83s/it] 


Сохранено: tpot_pipeline.joblib


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
# Оценка качества
y_pred = tpot.predict(X_test)
test_pr = accuracy_score(y_test, y_pred)
print(f"Точность на тестовой выборке: {test_pr:.4f}")

# Лучшие параметры и конвейер
print("\nЛучший конвейер:")
print(tpot.fitted_pipeline_)


Точность на тестовой выборке: 0.9995

Лучший конвейер:
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('selectfwe', SelectFwe(alpha=0.0012903345738)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('featureunion',
                                                 FeatureUnion(transformer_list=[('columnordinalencoder',
                                                                                 ColumnOrdinalEncoder()),
                                                                                ('binarizer',
                                                                                 Binarizer(threshold=0.55347461412))])),
                                                ('passthrough',
                                                 Passthrough())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer(

# 3. Сравнение результатов

In [ ]:
import pandas as pd
from IPython.display import display

# Данные из твоего сообщения
rows = [
    {
        "Метод": "Grid Search",
        "Лучшие параметры": {
            "classifier__criterion": "entropy",
            "classifier__max_depth": None,
            "classifier__max_features": "sqrt",
            "classifier__min_samples_leaf": 1,
            "classifier__min_samples_split": 2,
        },
        "PR-AUC (CV)": 0.8472271602900449,
        "ROC-AUC (CV)": 0.9478521341480878,
        "Accuracy (тест)": 0.8802190108930732,
    },
    {
        "Метод": "Random Search",
        "Лучшие параметры": {
            "classifier__criterion": "entropy",
            "classifier__max_depth": None,
            "classifier__max_features": "sqrt",
            "classifier__min_samples_leaf": 1,
            "classifier__min_samples_split": 2,
        },
        "PR-AUC (CV)": 0.8472271602900449,
        "ROC-AUC (CV)": None,
        "Accuracy (тест)": 0.8802190108930732,
    },
    {
        "Метод": "Optuna",
        "Лучшие параметры": {
            "classifier__criterion": "entropy",
            "classifier__max_depth": 5,
            "classifier__min_samples_leaf": 3,
            "classifier__min_samples_split": 3,
        },
        "PR-AUC (CV)": 0.7946922865296099,
        "ROC-AUC (CV)": None,
        "Accuracy (тест)": 0.9983322214809873,
    },
]

df = pd.DataFrame(rows)[["Метод", "Лучшие параметры",
                         "PR-AUC (CV)", "ROC-AUC (CV)", "Accuracy (тест)"]]
df_sorted = df.sort_values(
    by="Accuracy (тест)", ascending=False).reset_index(drop=True)

display(df_sorted)


,Метод,Лучшие параметры,PR-AUC (CV),ROC-AUC (CV),Accuracy (тест)
0,Optuna,"{'classifier__criterion': 'entropy', 'classifi...",0.794692,NaN,0.998332
1,Grid Search,"{'classifier__criterion': 'entropy', 'classifi...",0.847227,0.947852,0.880219
2,Random Search,"{'classifier__criterion': 'entropy', 'classifi...",0.847227,NaN,0.880219
